# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 20 2023 10:35AM,261490,10,PRF-58800,Bio-PRF,Released
1,Apr 20 2023 10:35AM,261490,10,PRF-58803,Bio-PRF,Released
2,Apr 20 2023 10:35AM,261490,10,PRF-58806,Bio-PRF,Released
3,Apr 20 2023 10:35AM,261490,10,PRF-58809,Bio-PRF,Released
4,Apr 20 2023 10:35AM,261490,10,PRF-58812,Bio-PRF,Released
5,Apr 20 2023 10:35AM,261490,10,PRF-58818,Bio-PRF,Released
6,Apr 20 2023 10:35AM,261490,10,PRF-58858,Bio-PRF,Released
7,Apr 20 2023 10:35AM,261490,10,PRF-58862,Bio-PRF,Released
8,Apr 20 2023 10:35AM,261490,10,PRF-58865,Bio-PRF,Released
9,Apr 20 2023 10:35AM,261490,10,PRF-58828,Bio-PRF,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
22,261486,Executing,1
23,261487,Released,1
24,261488,Released,12
25,261489,Released,1
26,261490,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261486,NaN,1.0,NaN
261487,NaN,NaN,1.0
261488,NaN,NaN,12.0
261489,NaN,NaN,1.0
261490,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261400,0.0,14.0,28.0
261405,0.0,0.0,2.0
261406,0.0,1.0,0.0
261410,2.0,2.0,5.0
261432,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261400,0,14,28
261405,0,0,2
261406,0,1,0
261410,2,2,5
261432,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261400,0,14,28
1,261405,0,0,2
2,261406,0,1,0
3,261410,2,2,5
4,261432,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261400,,14,28
1,261405,,,2
2,261406,,1,
3,261410,2,2,5
4,261432,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 20 2023 10:35AM,261490,10,Bio-PRF
12,Apr 20 2023 10:33AM,261488,10,Eywa Pharma Inc.
24,Apr 20 2023 10:32AM,261489,19,"Graystone, LLC"
25,Apr 20 2023 10:29AM,261487,19,"Graystone, LLC"
26,Apr 20 2023 10:28AM,261486,21,"NBTY Global, Inc."
27,Apr 20 2023 10:26AM,261485,19,"Methapharm, Inc."
30,Apr 20 2023 10:16AM,261484,19,"Emersa Waterbox, LLC"
31,Apr 20 2023 9:45AM,261448,15,"Person & Covey, Inc."
36,Apr 20 2023 9:42AM,261481,19,"Dr. Raju's Pharma Solutions, LLC"
37,Apr 20 2023 9:37AM,261400,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 20 2023 10:35AM,261490,10,Bio-PRF,,,12
1,Apr 20 2023 10:33AM,261488,10,Eywa Pharma Inc.,,,12
2,Apr 20 2023 10:32AM,261489,19,"Graystone, LLC",,,1
3,Apr 20 2023 10:29AM,261487,19,"Graystone, LLC",,,1
4,Apr 20 2023 10:28AM,261486,21,"NBTY Global, Inc.",,1,
5,Apr 20 2023 10:26AM,261485,19,"Methapharm, Inc.",,,3
6,Apr 20 2023 10:16AM,261484,19,"Emersa Waterbox, LLC",,,1
7,Apr 20 2023 9:45AM,261448,15,"Person & Covey, Inc.",,,5
8,Apr 20 2023 9:42AM,261481,19,"Dr. Raju's Pharma Solutions, LLC",,,1
9,Apr 20 2023 9:37AM,261400,15,"Alliance Pharma, Inc.",,14,28


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 10:35AM,261490,10,Bio-PRF,12,,
1,Apr 20 2023 10:33AM,261488,10,Eywa Pharma Inc.,12,,
2,Apr 20 2023 10:32AM,261489,19,"Graystone, LLC",1,,
3,Apr 20 2023 10:29AM,261487,19,"Graystone, LLC",1,,
4,Apr 20 2023 10:28AM,261486,21,"NBTY Global, Inc.",,1,
5,Apr 20 2023 10:26AM,261485,19,"Methapharm, Inc.",3,,
6,Apr 20 2023 10:16AM,261484,19,"Emersa Waterbox, LLC",1,,
7,Apr 20 2023 9:45AM,261448,15,"Person & Covey, Inc.",5,,
8,Apr 20 2023 9:42AM,261481,19,"Dr. Raju's Pharma Solutions, LLC",1,,
9,Apr 20 2023 9:37AM,261400,15,"Alliance Pharma, Inc.",28,14,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 10:35AM,261490,10,Bio-PRF,12,,
1,Apr 20 2023 10:33AM,261488,10,Eywa Pharma Inc.,12,,
2,Apr 20 2023 10:32AM,261489,19,"Graystone, LLC",1,,
3,Apr 20 2023 10:29AM,261487,19,"Graystone, LLC",1,,
4,Apr 20 2023 10:28AM,261486,21,"NBTY Global, Inc.",,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 10:35AM,261490,10,Bio-PRF,12.0,NaN,NaN
1,Apr 20 2023 10:33AM,261488,10,Eywa Pharma Inc.,12.0,NaN,NaN
2,Apr 20 2023 10:32AM,261489,19,"Graystone, LLC",1.0,NaN,NaN
3,Apr 20 2023 10:29AM,261487,19,"Graystone, LLC",1.0,NaN,NaN
4,Apr 20 2023 10:28AM,261486,21,"NBTY Global, Inc.",NaN,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 20 2023 10:35AM,261490,10,Bio-PRF,12.0,0.0,0.0
1,Apr 20 2023 10:33AM,261488,10,Eywa Pharma Inc.,12.0,0.0,0.0
2,Apr 20 2023 10:32AM,261489,19,"Graystone, LLC",1.0,0.0,0.0
3,Apr 20 2023 10:29AM,261487,19,"Graystone, LLC",1.0,0.0,0.0
4,Apr 20 2023 10:28AM,261486,21,"NBTY Global, Inc.",0.0,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1307402,31.0,0.0,0.0
15,522848,33.0,14.0,0.0
19,3137566,97.0,9.0,0.0
20,522816,5.0,3.0,2.0
21,261486,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1307402,31.0,0.0,0.0
1,15,522848,33.0,14.0,0.0
2,19,3137566,97.0,9.0,0.0
3,20,522816,5.0,3.0,2.0
4,21,261486,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,0.0,0.0
1,15,33.0,14.0,0.0
2,19,97.0,9.0,0.0
3,20,5.0,3.0,2.0
4,21,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,15,Released,33.0
2,19,Released,97.0
3,20,Released,5.0
4,21,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21
Status,,,,,
Completed,0.0,0.0,0.0,2.0,0.0
Executing,0.0,14.0,9.0,3.0,1.0
Released,31.0,33.0,97.0,5.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21
0,Completed,0.0,0.0,0.0,2.0,0.0
1,Executing,0.0,14.0,9.0,3.0,1.0
2,Released,31.0,33.0,97.0,5.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21
0,Completed,0.0,0.0,0.0,2.0,0.0
1,Executing,0.0,14.0,9.0,3.0,1.0
2,Released,31.0,33.0,97.0,5.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()